# COMP5046 Assignment 1
*Make sure you change the file name with your unikey.*

# Readme
*If there is something to be noted for the user, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style*

[link text](https://)***Visualising the comparison of different results is a good way to justify your decision.***

# 1 - Data Preprocessing

## 1.1. Download Dataset

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1vF3FqgBC1Y-RPefeVmY8zetdZG1jmHzT'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('imdb_train.csv')

id = '1XhaV8YMuQeSwozQww8PeyiWMJfia13G6'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('imdb_test.csv')

import pandas as pd
df_train = pd.read_csv("imdb_train.csv")
df_test = pd.read_csv("imdb_test.csv")

reviews_train = df_train['review'].tolist()
sentiments_train = df_train['sentiment'].tolist()
reviews_test = df_test['review'].tolist()
sentiments_test = df_test['sentiment'].tolist()

print("Training set number:",len(reviews_train))
print("Testing set number:",len(reviews_test))

Training set number: 25000
Testing set number: 25000


## 1.2. Preprocess data

*Pre-processing techniques:*

1.   removing numbers
2.   converting to lowercase
3.   removing stop words
5.   tokenizer.TweetTokenizer is applied as it can extract url
6.   removing html tag like \<br /\>
7.   expanding contraction


In [ ]:
# import necessary libraries
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
from nltk.stem.porter import *
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import numpy as np
from sklearn.metrics import accuracy_score,f1_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from collections import Counter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}


In [ ]:
def multiple_replace(text,ad):
    """
    This function is used to replace multiple string at a time and accept a dict as mapping
    """
    rx=re.compile('|'.join([re.escape(x) for x in ad.keys()]))
    def one_xlat(match):
        return ad[match.group(0)]
    return rx.sub(one_xlat,text)


tknzr=TweetTokenizer()
tag_dict={'<br />':''}
stopwords_dict={word:'' for word in sw.words()}
dicts={**contraction_dict, **tag_dict}
reviews_train = [multiple_replace(text.lower(),dicts) for text in reviews_train]
reviews_train = [list(filter(lambda x: re.fullmatch(r'[a-z\']+',x) #this regular expression is to remove punctuation, url and keep possessive
                               and x not in stopwords_dict #remove stop words
                               ,tknzr.tokenize(text)))
                for text in reviews_train              
          ]           

reviews_test=[multiple_replace(text.lower(),dicts) for text in reviews_test]
reviews_test=[list(filter(lambda x: re.fullmatch(r'[a-z\']+',x) 
                               and x not in stopwords_dict
                               ,tknzr.tokenize(text)))
              for text in reviews_test              
          ]



In [ ]:
# encode labels to 0-negtive and 1-positive
labels = np.unique(sentiments_train)
lEnc = LabelEncoder()
lEnc.fit(labels)
label_train_n = lEnc.transform(sentiments_train)
label_test_n = lEnc.transform(sentiments_test)
numClass = len(labels)

print(labels)
print(lEnc.transform(labels))

['neg' 'pos']
[0 1]


# 2 - Model Implementation

## 2.1. Word Embeddings

Word2Vec with Skip-gram is implemented. Word2Vec with Skip-gram normally has higher accuracy

### 2.1.1. Data Preprocessing for Word Embeddings

This section uses [reviews_train] from section 1 

and <b>stemming</b> is implemented  as lemmatisation does not work well when parts of speech of words are not given.

Besides,the vocabulary size after stemming is less than lemmatisation,which can reduce training time

In [ ]:
# convert all sentences to unique word list. 
stemmer = PorterStemmer()
word_sequence = [stemmer.stem(word) for reviews in reviews_train for word in reviews]
word_list = [stemmer.stem(word) for reviews in reviews_train for word in reviews]
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}


skip_grams = []

for i in range(1, len(word_sequence) - 1):
    target = word_dict[word_sequence[i]]
    context = [word_dict[word_sequence[i - 1]], word_dict[word_sequence[i + 1]]]

    for w in context:
        skip_grams.append([target, w])
def prepare_batch(data, size):
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(data)), size, replace=False)

    for i in random_index:
        input_temp = [0]*voc_size
        input_temp[data[i][0]] = 1
        random_inputs.append(input_temp)  
        random_labels.append(data[i][1])  

    return np.array(random_inputs), np.array(random_labels)

In [ ]:
print(len(word_list))
print(len(skip_grams))

55198
5616570


### 2.1.2. Build Word Embeddings Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [ ]:
# Setting hyperparameters
# when learning_rate equals to 0.1, the training loss is very unstable
# when learning_rate equals to 0.01, the training loss descreases stably
# embedding_size=50 won't blow up RAM memory
voc_size = len(word_list)
learning_rate = 0.01
batch_size = 100
embedding_size = 50
epochs=5000

In [ ]:
class SkipGram(nn.Module):
    def __init__(self):
        super(SkipGram, self).__init__()
        self.linear1 = nn.Linear(voc_size, embedding_size,bias=False)
        self.linear2 = nn.Linear(embedding_size, voc_size,bias=False)

    def forward(self, x):
        hidden = self.linear1(x)
        out = self.linear2(hidden)
        return out
skip_gram_model = SkipGram().to(device)
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(skip_gram_model.parameters(), lr=learning_rate)

### 2.1.3. Train Word Embeddings Model

In [ ]:
for epoch in range(epochs):
    inputs,labels = prepare_batch(skip_grams, batch_size)
    inputs_torch = torch.from_numpy(inputs).float().to(device)
    labels_torch = torch.from_numpy(labels).to(device)
    
    
    skip_gram_model.train()
    # 1. zero grad
    optimiser.zero_grad()
    # 2. forword propagation
    outputs=skip_gram_model(inputs_torch)
    # 3. calculate loss
    loss=criterion(outputs,labels_torch)
    # 4. back propagation
    loss.backward()
    optimiser.step()
    if epoch % 500 == 499: 
        print('Epoch: %d, loss: %.4f' %(epoch + 1, loss))

Epoch: 500, loss: 8.9848
Epoch: 1000, loss: 9.1340
Epoch: 1500, loss: 8.7943
Epoch: 2000, loss: 8.6207
Epoch: 2500, loss: 8.9641
Epoch: 3000, loss: 8.6814
Epoch: 3500, loss: 8.1079
Epoch: 4000, loss: 8.0993
Epoch: 4500, loss: 8.3282
Epoch: 5000, loss: 8.2813


### 2.1.4. Save Word Embeddings Model

In [ ]:
# Save model and upload to Google Drive
torch.save(skip_gram_model, 'lewu9692_word.pt')
uploaded = drive.CreateFile({'title': 'lewu9692_word.pt'})
uploaded.SetContentFile('lewu9692_word.pt')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type SkipGram. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Uploaded file with ID 1yOjyjEU6AInUXeaObPalPNlinIuxYZ-z


### 2.1.5. Load Word Embeddings Model

In [ ]:
# download model from Google Drive and load it 
# to load model successfully, you need to rerun the cells in this section except for training part.
id = '1yOjyjEU6AInUXeaObPalPNlinIuxYZ-z'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('lewu9692_word.pt')
word_embedding_model = torch.load('lewu9692_word.pt').to(device)

## 2.2. Character Embeddings

### 2.2.1. Data Preprocessing for Character Embeddings

This section uses same data from [word embedding] section

In [ ]:
# this section uses the same vocabulary as word embedding section

# extract weight from word embedding model
weight1 = word_embedding_model.linear1.weight
word_embeddings = weight1.detach().T.cpu().numpy()

#Assume that we have the following character instances
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',

            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z','\'']

# one-hot encoding and decoding 
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n: i+1 for i, n in enumerate(char_arr)}
num_dic['P'] = 0 #encoding for padding
dic_len = len(num_dic)

In [ ]:
# there are some very long words but they are minority.
# if padding with max length of words, input of model will be huge.
# To find a proper length truncating, accumalate amount of each length until it accounts for defined ratio
word_length_cnt=Counter(map(len,word_list)).most_common()
word_length_cnt.sort()
total_amt=0
ratio=0.95
for idx,(length,amt) in enumerate(word_length_cnt):
  if total_amt/len(word_list)<=ratio:
    total_amt += amt
  else:
    break
max_word_len = idx
max_word_len

11

In [ ]:
def add_padding(word):
  if len(word)>=max_word_len:
    return word[:max_word_len]
  else:
    return word+'P'*(max_word_len-len(word))
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        after_padding=add_padding(seq)
        input_data = [num_dic[n] for n in after_padding]
        target = word_embeddings[seq_data.index(seq)]
        # convert input to one-hot encoding.
        # if input is [3, 4, 4]:
        # [[ 0,  0,  0,  1,  0,  0,  0, ... 0]
        #  [ 0,  0,  0,  0,  1,  0,  0, ... 0]
        #  [ 0,  0,  0,  0,  1,  0,  0, ... 0]]
        input_batch.append(np.eye(dic_len)[input_data])
        
        target_batch.append(target)

    return input_batch, target_batch

### 2.2.2. Build Character Embeddings Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [ ]:
# setting hyperparameters
# from previous experience, learning_rate more than 0.01 may result in a big loss

learning_rate = 0.01
n_hidden = 128
total_epoch = 5000
n_input = dic_len
n_class = embedding_size

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(n_input, n_hidden, batch_first =True,bidirectional=True, dropout=0.2)
        self.linear = nn.Linear(n_hidden*2,n_class)

    def forward(self, sentence):
        
        #h_n of shape (num_layers * num_directions, batch, hidden_size): tensor containing the hidden state for t = seq_len.
        lstm_out, (h_n,c_n) = self.lstm(sentence)
        #concat the last hidden state from two direction
        hidden_out =torch.cat((h_n[0,:,:],h_n[1,:,:]),1)
        z = self.linear(hidden_out)
        log_output = F.log_softmax(z, dim=1)
        return log_output,hidden_out


### 2.1.4. Train Character Embeddings Model

In [ ]:
# Preparing input
input_batch, target_batch = make_batch(word_list)
# Convert input into tensors and move them to GPU by uting tensor.to(device)
input_batch_torch = torch.from_numpy(np.array(input_batch)).float().to(device)
target_batch_torch = torch.from_numpy(np.array(target_batch)).to(device)


In [ ]:
# Move the model to GPU
net = Net().to(device)
# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
for epoch in range(total_epoch):  
    
    # Set the flag to training
    net.train()
    # forward + backward + optimize
    outputs,_ = net(input_batch_torch) 
    loss = criterion(outputs, target_batch_torch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    # Set the flag to evaluation, which will 'turn off' the dropout
    net.eval()
    outputs,_ = net(input_batch_torch) 
    # Evaluation loss and accuracy calculation
    loss = criterion(outputs, target_batch_torch)
    if epoch % 500 == 499:
      print('Epoch: %d, loss: %.5f' %(epoch + 1, loss.item()))

print('Finished Training')

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 500, loss: 15.47548
Epoch: 1000, loss: 15.47514
Epoch: 1500, loss: 15.47333
Epoch: 2000, loss: 15.47294
Epoch: 2500, loss: 15.47261
Epoch: 3000, loss: 15.47240
Epoch: 3500, loss: 15.47225
Epoch: 4000, loss: 15.47215
Epoch: 4500, loss: 15.47206
Epoch: 5000, loss: 15.47199
Finished Training


### 2.1.5. Save Character Embeddings Model

In [ ]:
#save model and upload it to Google Drive
torch.save(net, 'lewu9692_character.pt')

uploaded = drive.CreateFile({'title': 'lewu9692_character.pt'})
uploaded.SetContentFile('lewu9692_character.pt')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Uploaded file with ID 1m8FAdDTuizqTpNzGbbzDHn5ftarvhyQE


### 2.1.6. Load Character Embeddings Model

In [ ]:
# download model from Google Drive and load it
# to load model successfully, you need to rerun the cells in this section except for training part.
id = '1m8FAdDTuizqTpNzGbbzDHn5ftarvhyQE'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('lewu9692_character.pt')
character_embedding_model = torch.load('lewu9692_character.pt')


## 2.3. Sequence model

### 2.3.1. Apply/Import Word Embedding and Character Embedding Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [ ]:
# there are some very long reviews but they are minority.
# if padding with max length of reviews, input of model will be huge.
# To find a proper length truncating, accumalate amount of each length until it accounts for defined ratio of total amount of reviews
from collections import Counter
cnt=Counter(map(len,reviews_train)).most_common()
cnt.sort()

total_amt=0
ratio=0.8
for idx,(length,amt) in enumerate(cnt):
  if total_amt/len(reviews_train)<=ratio:
    total_amt += amt
  else:
    break
max_review_len = idx
max_review_len

153

In [ ]:
# extract hidding state from character embedding model
character_embedding_model.eval()
_,hidden_state = character_embedding_model(input_batch_torch)
character_embeddings = hidden_state.data

# extract weight from word embedding model
word_embedding_model.eval()
weight1 = word_embedding_model.linear1.weight
word_embeddings = weight1.detach().T

# concatenate word embeddings and character embeddings
embeddings = torch.cat((word_embeddings,character_embeddings),1).cpu().numpy()


In [ ]:
# this function is to get embedding of reviews from concatenated embeddings
def generate_embeddings(sentences,embeddings):

  input_embeddings = []
  zero_padding = np.zeros((embeddings.shape[1],))
  for sentence in tqdm(sentences):
    tmp_embeddings = []
    for idx in range(max_review_len):
      try:
        tmp_embeddings.append(embeddings[word_list.index(sentence[idx])])
      except:
        tmp_embeddings.append(zero_padding)
    input_embeddings.append(tmp_embeddings)
  return np.array(input_embeddings)

In [ ]:
# please note that this step will cost one hours
train_embeddings = generate_embeddings(reviews_train,embeddings)
train_target = torch.from_numpy(np.array(label_train_n)).to(device)
train_embeddings_torch = torch.from_numpy(train_embeddings).float().to(device) # move embeddings to GPU

### 2.3.2. Build Sequence Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [ ]:
# according to the figure in the [evaluation] section, when learning rate equals to 0.01, the model performs better.
# and at epoch 1000, f1 score remains stable.
n_input = train_embeddings_torch.shape[2]
n_hidden = 50
n_class = len(labels)
total_epoch = 1000
learning_rate = 0.01
train_batch_size = 5000

In [ ]:
# prepare batch of training embedding
train_data = data.TensorDataset(train_embeddings_torch,train_target)
train_loader = data.DataLoader(dataset=train_data,batch_size=train_batch_size,shuffle=True)

In [ ]:
class SeqNet(nn.Module):
    def __init__(self):
        super(SeqNet, self).__init__()
        self.lstm = nn.LSTM(n_input, n_hidden, num_layers=2, batch_first =True, dropout=0.2)
        self.linear = nn.Linear(n_hidden,n_class)

    def forward(self, x):
        x,_ = self.lstm(x)
        x = self.linear(x[:,-1,:])
        x = F.log_softmax(x, dim=1)
        return x

### 2.3.3. Train Sequence Model

In [ ]:
seq = SeqNet().to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(seq.parameters(), lr=learning_rate)
for epoch in range(total_epoch):  
    for batch,(train_embeddings_from_loader,train_target_from_loader) in enumerate(train_loader):
      seq.train()
      outputs = seq(train_embeddings_from_loader) 
      loss = criterion(outputs, train_target_from_loader)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      seq.eval()
      outputs = seq(train_embeddings_from_loader) 
      if epoch%100 == 99 and batch == int(train_embeddings.shape[0]/train_batch_size)-1:
          loss = criterion(outputs, train_target_from_loader)
          _, predicted = torch.max(outputs, 1)
          acc= accuracy_score(train_target_from_loader.cpu().numpy(),predicted.cpu().numpy())
          f1= f1_score(train_target_from_loader.cpu().numpy(),predicted.cpu().numpy())
          print('Epoch: %d, loss: %.5f, train_acc: %.4f, train_f1_score: %.4f' %(epoch + 1, loss.item(), acc, f1))

print('Finished Training')

Epoch: 100, loss: 0.69402, train_acc: 0.4960, train_f1_score: 0.0000
Epoch: 200, loss: 0.69181, train_acc: 0.5004, train_f1_score: 0.6670
Epoch: 300, loss: 0.40075, train_acc: 0.8264, train_f1_score: 0.8148
Epoch: 400, loss: 0.19478, train_acc: 0.9420, train_f1_score: 0.9417
Epoch: 500, loss: 0.13474, train_acc: 0.9660, train_f1_score: 0.9662
Epoch: 600, loss: 0.10610, train_acc: 0.9740, train_f1_score: 0.9742
Epoch: 700, loss: 0.09108, train_acc: 0.9774, train_f1_score: 0.9773
Epoch: 800, loss: 0.38112, train_acc: 0.8268, train_f1_score: 0.7933
Epoch: 900, loss: 0.04826, train_acc: 0.9888, train_f1_score: 0.9888
Epoch: 1000, loss: 0.03697, train_acc: 0.9902, train_f1_score: 0.9903
Finished Training


### 2.3.4. Save Sequence Model

In [ ]:
# save model and upload it to Google Drive
torch.save(seq, 'lewu9692_sequence.pt')

uploaded = drive.CreateFile({'title': 'lewu9692_sequence.pt'})
uploaded.SetContentFile('lewu9692_sequence.pt')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning:

Couldn't retrieve source code for container of type SeqNet. It won't be checked for correctness upon loading.



Uploaded file with ID 1NDzReLC6Ozn8cUK8LqVUv4Ipgw5zn_gN


### 2.3.5. Load Sequence Model

In [ ]:
# download model from Google Drive and load it
id = '1NDzReLC6Ozn8cUK8LqVUv4Ipgw5zn_gN'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('lewu9692_sequence.pt')
sequence_model = torch.load('lewu9692_sequence.pt')

# 3 - Evaluation

(*Please show your empirical evidence*)

## 3.1. Performance Evaluation


You are required to provide the table with precision, recall, f1 of test set.

In [ ]:
# prepare test embeddings. Please note that this step will cost one hour
test_embeddings = generate_embeddings(reviews_test,embeddings)
test_target = torch.from_numpy(np.array(label_test_n)).to(device)

In [ ]:
# predict test embeddings in batch to prevent CUDA out of memory
test_batch_size = 5000
predictions = []
for i in np.arange(0,test_embeddings.shape[0],test_batch_size):
  test_embeddings_torch = torch.from_numpy(test_embeddings[i:i+test_batch_size]).float().to(device)
  seq.eval()
  outputs = seq(test_embeddings_torch) 
  _, predicted = torch.max(outputs, 1)
  predictions += list(predicted.cpu().numpy())

predictions = np.array(predictions)
from sklearn.metrics import classification_report
print(classification_report(label_test_n,predictions))


              precision    recall  f1-score   support

           0       0.74      0.77      0.76     12500
           1       0.76      0.73      0.75     12500

    accuracy                           0.75     25000
   macro avg       0.75      0.75      0.75     25000
weighted avg       0.75      0.75      0.75     25000



## 3.2. Hyperparameter Testing
*You are required to draw a graph(y-axis: f1, x-axis: epoch) for test set and explain the optimal number of epochs based on the learning rate you have already chosen.*

In [ ]:
# as the following displays, the model with learning rate=0.01 is more stable than other two 
# and its f1 score is relatively higher at epoch 1000.
total_epoch_test=2000
test_embeddings_parts = torch.from_numpy(test_embeddings[:1000]).float().to(device) #split 1000 batches for hyperparameter testing
test_target_parts = test_target[:1000]
hypers = {'lr':[],'epoch':[],'f1':[]}
for learning_rate in [0.1,0.01,0.001]:
  seq = SeqNet().to(device) #initialize model
  criterion = nn.NLLLoss()
  optimizer = optim.Adam(seq.parameters(), lr=learning_rate)
  for epoch in range(total_epoch_test):  
      for batch,(train_embeddings_from_loader,train_target_from_loader) in enumerate(train_loader):
        seq.train()
        outputs = seq(train_embeddings_from_loader) 
        loss = criterion(outputs, train_target_from_loader)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        seq.eval()
        outputs = seq(test_embeddings_parts) 
        if epoch%50 == 49 and batch == int(train_embeddings.shape[0]/train_batch_size)-1:
            loss = criterion(outputs, test_target_parts)
            _, predicted = torch.max(outputs, 1)
            acc= accuracy_score(test_target_parts.cpu().numpy(),predicted.cpu().numpy())
            f1= f1_score(test_target_parts.cpu().numpy(),predicted.cpu().numpy())
            hypers['lr'].append(learning_rate)
            hypers['epoch'].append(epoch)
            hypers['f1'].append(f1)
            print('lr: %.3f, Epoch: %d, loss: %.5f, train_acc: %.4f, train_f1_score: %.4f' %(learning_rate,epoch + 1, loss.item(), acc, f1))

print('Finished Training')

lr: 0.100, Epoch: 50, loss: 0.75798, train_acc: 0.7280, train_f1_score: 0.7349
lr: 0.100, Epoch: 100, loss: 0.82571, train_acc: 0.7290, train_f1_score: 0.7451
lr: 0.100, Epoch: 150, loss: 0.78348, train_acc: 0.7230, train_f1_score: 0.7287
lr: 0.100, Epoch: 200, loss: 0.78476, train_acc: 0.7350, train_f1_score: 0.7347
lr: 0.100, Epoch: 250, loss: 0.70280, train_acc: 0.7400, train_f1_score: 0.7426
lr: 0.100, Epoch: 300, loss: 0.63881, train_acc: 0.6400, train_f1_score: 0.6797
lr: 0.100, Epoch: 350, loss: 0.61520, train_acc: 0.6510, train_f1_score: 0.7197
lr: 0.100, Epoch: 400, loss: 0.58936, train_acc: 0.6910, train_f1_score: 0.7147
lr: 0.100, Epoch: 450, loss: 0.59130, train_acc: 0.6760, train_f1_score: 0.7282
lr: 0.100, Epoch: 500, loss: 0.56660, train_acc: 0.7070, train_f1_score: 0.7196
lr: 0.100, Epoch: 550, loss: 0.55281, train_acc: 0.7090, train_f1_score: 0.7215
lr: 0.100, Epoch: 600, loss: 0.56558, train_acc: 0.7130, train_f1_score: 0.7139
lr: 0.100, Epoch: 650, loss: 0.55144, tra

In [ ]:
import plotly.express as px
df = pd.DataFrame(hypers)
fig = px.line(df, x="epoch", y="f1", color='lr')
fig.show()

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed* 

In [ ]:
# If you used OOP style, use this section